In [1]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
from scipy import stats
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
from datetime import datetime
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm
from IPython.display import display, HTML
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from collections import Counter
import string
from wordcloud import WordCloud
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: unset; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#loading the dataset
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)

df_trustpilot = pd.read_csv('data/data_trustpilot_2.csv', engine='python')

display(df_trustpilot.head())



,rating,location,username,number_reviews,verification,repeat_reviewer,repeat_reviewer_encoded,company,text,text_processed,text_length,text_word_length,Sentiment_Blob,Sentiment_Blob_cat,Sentiment_VADER,Sentiment_VADER_cat,subject,subject_processed,subject_length,subject_word_length,answer,answer_processed,answered_encoded,date_of_experience,date_posted,local_date_posted,month_local,local_hour,time_of_day,day_of_week_posted,day_type,days_between_experience_and_post,review_time,review_time_encoded
0,5,GB,Dest Iny,7,Not Verified,repeat,1,Asda,"ST2 9AL Had an issue with the scan, an employe...",st2 9al issue scan employee named rose helped ...,215,35,0.435000,4.0,0.8316,5,ST2 9AL,st2 9al,7,2,0,0,0,2024-11-11,2024-11-15 11:42:09,2024-11-15,11,10,Business Hours,4,Business Day,4,late_review,0
1,1,GB,Paula Gray,4,Not Verified,repeat,1,Asda,Your call centre staff are not the most helpfu...,call centre staff helpful several times tried ...,265,39,-0.228571,2.0,-0.5859,2,Your call centre staff are not the most…,call centre staff,17,3,0,0,0,2024-11-13,2024-11-13 18:26:52,2024-11-13,11,17,Business Hours,2,Business Day,0,quick_review,1
2,1,GB,Neil Dodds,4,Not Verified,repeat,1,Asda,"Just received my order from asda accrington, ...",received order asda accrington ordered 2 rump ...,236,36,0.250000,4.0,0.4939,4,Just received my order from asda…,received order asda,19,3,0,0,0,2024-11-14,2024-11-14 21:20:00,2024-11-14,11,20,Evening,3,Business Day,0,quick_review,1
3,1,GB,Julie Currey,5,Not Verified,repeat,1,Asda,"Just had an online home delivery dropped off, ...",online home delivery dropped substitutes items...,370,56,-0.275000,2.0,-0.9540,1,"Asda, where did it all go wrong?",asda go wrong,13,3,0,0,0,2024-11-13,2024-11-13 23:34:32,2024-11-13,11,22,Evening,2,Business Day,0,quick_review,1
4,2,GB,Jessiejames,3,Not Verified,repeat,1,Asda,Used the Asda rewards app to save for Christma...,used asda rewards app save christmas noticed p...,152,23,-0.033333,3.0,0.9071,5,Rewards not worth it,rewards worth,13,2,0,0,0,2024-10-01,2024-11-10 22:10:35,2024-11-10,11,21,Evening,6,Weekend,40,late_review,0


In [3]:
#First check
df_trustpilot['local_date_posted'] = pd.to_datetime(df_trustpilot['local_date_posted'])
df_trustpilot['date_of_experience'] = pd.to_datetime(df_trustpilot['date_of_experience'])
df_trustpilot['date_posted'] = pd.to_datetime(df_trustpilot['date_posted'])
print("Null values 1: ", df_trustpilot.isna().sum().sum())
df_trustpilot = df_trustpilot.dropna(how='any')
print("Null values 2: ", df_trustpilot.isna().sum().sum())
print("Duplicates: ", df_trustpilot.duplicated().sum())

print(df_trustpilot.columns)
display(df_trustpilot.info())

Null values 1:  359
Null values 2:  0
Duplicates:  0
Index(['rating', 'location', 'username', 'number_reviews', 'verification',
       'repeat_reviewer', 'repeat_reviewer_encoded', 'company', 'text',
       'text_processed', 'text_length', 'text_word_length', 'Sentiment_Blob',
       'Sentiment_Blob_cat', 'Sentiment_VADER', 'Sentiment_VADER_cat',
       'subject', 'subject_processed', 'subject_length', 'subject_word_length',
       'answer', 'answer_processed', 'answered_encoded', 'date_of_experience',
       'date_posted', 'local_date_posted', 'month_local', 'local_hour',
       'time_of_day', 'day_of_week_posted', 'day_type',
       'days_between_experience_and_post', 'review_time',
       'review_time_encoded'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 100262 entries, 0 to 100620
Data columns (total 34 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   

None

STAGE 1 - TRAIN/TEST SPLIT

In [4]:
# Splitting data with stratification on rating to make sure that ratings are distributed evenly across both sets due to data imbalance
train_df, test_df = train_test_split(df_trustpilot, test_size=0.2,  
    random_state=42, stratify=df_trustpilot['rating'])

print("Training set size:", len(train_df))
print("Test set size:", len(test_df))

# Verifying rating distribution
print("Rating distribution in original data:")
print(df_trustpilot['rating'].value_counts(normalize=True).round(3))
print("Rating distribution in training set:")
print(train_df['rating'].value_counts(normalize=True).round(3))
print("Rating distribution in test set:")
print(test_df['rating'].value_counts(normalize=True).round(3))

Training set size: 80209
Test set size: 20053
Rating distribution in original data:
rating
5    0.466
1    0.240
4    0.123
3    0.099
2    0.072
Name: proportion, dtype: float64
Rating distribution in training set:
rating
5    0.466
1    0.240
4    0.123
3    0.099
2    0.072
Name: proportion, dtype: float64
Rating distribution in test set:
rating
5    0.466
1    0.240
4    0.123
3    0.099
2    0.072
Name: proportion, dtype: float64


STAGE 2 - HANDLE OUTLIERS

In [5]:
# Handle outliers on numeric columns that make sense
numeric_cols = ['number_reviews', 'days_between_experience_and_post',
                'text_length', 'text_word_length', 'subject_length', 
                'subject_word_length']

bounds = {}

# First pass: Calculate bounds from training data and handle training set outliers
for col in numeric_cols:
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    bounds[col] = {'lower': lower_bound, 'upper': upper_bound}
    
    outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)]
    print(f"{col} training outliers: {len(outliers)} ({(len(outliers)/len(train_df)*100):.2f}%)")
    
    # Clip training data
    train_df[col] = train_df[col].clip(lower_bound, upper_bound)

# Second pass: Handle test set outliers using bounds from training data
for col in numeric_cols:
    test_df[col] = test_df[col].clip(bounds[col]['lower'], bounds[col]['upper'])
    
    test_outliers = test_df[(test_df[col] < bounds[col]['lower']) | (test_df[col] > bounds[col]['upper'])]
    print(f"{col} test outliers handled: {len(test_outliers)} ({(len(test_outliers)/len(test_df)*100):.2f}%)")

#Evaluation to check output
print("Checking for remaining outliers:")
for col in numeric_cols:
    train_outliers = train_df[(train_df[col] < bounds[col]['lower']) | 
                             (train_df[col] > bounds[col]['upper'])]
    if len(train_outliers) > 0:
        print(f"Warning: {col} still has {len(train_outliers)} outliers in training set")
    else:
        print(f"{col}: No outliers remain in training set")
    
    test_outliers = test_df[(test_df[col] < bounds[col]['lower']) | 
                           (test_df[col] > bounds[col]['upper'])]
    if len(test_outliers) > 0:
        print(f"Warning: {col} still has {len(test_outliers)} outliers in test set")
    else:
        print(f"{col}: No outliers remain in test set")

number_reviews training outliers: 6885 (8.58%)
days_between_experience_and_post training outliers: 27301 (34.04%)
text_length training outliers: 5975 (7.45%)
text_word_length training outliers: 5919 (7.38%)
subject_length training outliers: 2158 (2.69%)
subject_word_length training outliers: 1292 (1.61%)
number_reviews test outliers handled: 0 (0.00%)
days_between_experience_and_post test outliers handled: 0 (0.00%)
text_length test outliers handled: 0 (0.00%)
text_word_length test outliers handled: 0 (0.00%)
subject_length test outliers handled: 0 (0.00%)
subject_word_length test outliers handled: 0 (0.00%)
Checking for remaining outliers:
number_reviews: No outliers remain in training set
number_reviews: No outliers remain in test set
days_between_experience_and_post: No outliers remain in training set
days_between_experience_and_post: No outliers remain in test set
text_length: No outliers remain in training set
text_length: No outliers remain in test set
text_word_length: No outlie

STAGE 3 - ADVANCED TEXT PREPROCESSING

In [6]:
# TF-IDF with unigrams and bigrams

def process_and_save_tfidf(train_df, test_df):
    tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
    max_df=0.8, min_df=5)

    # Fit and transform training data
    X_train_tfidf = tfidf.fit_transform(train_df['text_processed'])
    X_test_tfidf = tfidf.transform(test_df['text_processed'])
    # Get feature names (top n-grams)
    feature_names = tfidf.get_feature_names_out()

    np.save('data/X_train_tfidf.npy', X_train_tfidf.toarray())
    np.save('data/X_test_tfidf.npy', X_test_tfidf.toarray())
    np.save('data/tfidf_features.npy', tfidf.get_feature_names_out())

    print(f"Number of features: {len(feature_names)}")

    feature_importance = pd.DataFrame({'feature': feature_names,
        'importance': X_train_tfidf.sum(axis=0).A1}).sort_values('importance', ascending=False)

    print("Top 10 most important n-grams:")
    print(feature_importance.head(10))

    return {'tfidf_train_matrix': X_train_tfidf, 'tfidf_test_matrix': X_test_tfidf,
        'tfidf_features': feature_names}

results = process_and_save_tfidf(train_df, test_df)

Number of features: 5000
Top 10 most important n-grams:
       feature   importance
3825   service  1899.475074
1867     great  1819.886826
3357   quicken  1597.378816
1820      good  1596.231264
2144       ink  1566.655099
4609       use  1475.549477
4382      time  1322.780589
2904     order  1282.401797
1034  customer  1281.275110
1304      easy  1239.989153


In [7]:
#Bag-of-Words

# Bag of Words processing
def process_and_save_bow(train_df, test_df):
    count_vec = CountVectorizer(max_features=5000, ngram_range=(1, 2),
                               max_df=0.8, min_df=5)
    
    # Fit and transform training data, only transform test data
    X_train_bow = count_vec.fit_transform(train_df['text_processed'])
    X_test_bow = count_vec.transform(test_df['text_processed'])
    
    # Get feature names (vocabulary)
    feature_names = count_vec.get_feature_names_out()
    
    # Save the matrices and features
    np.save('data/X_train_bow.npy', X_train_bow.toarray())
    np.save('data/X_test_bow.npy', X_test_bow.toarray())
    np.save('data/bow_features.npy', feature_names)
    
    print(f"Number of features: {len(feature_names)}")
    print(f"Bag of Words Matrix Shape: {X_train_bow.shape}")
    print(f"Memory Usage: {X_train_bow.data.nbytes / (1024 ** 3):.2f} GB")
    
    return {'X_train_bow': X_train_bow,'X_test_bow': X_test_bow,'feature_names': feature_names}

results = process_and_save_bow(train_df, test_df)


Number of features: 5000
Bag of Words Matrix Shape: (80209, 5000)
Memory Usage: 0.01 GB


In [8]:
#Inspecting final output 

print(df_trustpilot.columns)
print(df_trustpilot.info())
print("The Trustpilot dataset has ", df_trustpilot.shape[1], "columns")
print("The Trustpilot dataset has ", df_trustpilot.shape[0], "rows")
display(df_trustpilot.head())

Index(['rating', 'location', 'username', 'number_reviews', 'verification',
       'repeat_reviewer', 'repeat_reviewer_encoded', 'company', 'text',
       'text_processed', 'text_length', 'text_word_length', 'Sentiment_Blob',
       'Sentiment_Blob_cat', 'Sentiment_VADER', 'Sentiment_VADER_cat',
       'subject', 'subject_processed', 'subject_length', 'subject_word_length',
       'answer', 'answer_processed', 'answered_encoded', 'date_of_experience',
       'date_posted', 'local_date_posted', 'month_local', 'local_hour',
       'time_of_day', 'day_of_week_posted', 'day_type',
       'days_between_experience_and_post', 'review_time',
       'review_time_encoded'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 100262 entries, 0 to 100620
Data columns (total 34 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   rating                            100262 non-null  in

,rating,location,username,number_reviews,verification,repeat_reviewer,repeat_reviewer_encoded,company,text,text_processed,text_length,text_word_length,Sentiment_Blob,Sentiment_Blob_cat,Sentiment_VADER,Sentiment_VADER_cat,subject,subject_processed,subject_length,subject_word_length,answer,answer_processed,answered_encoded,date_of_experience,date_posted,local_date_posted,month_local,local_hour,time_of_day,day_of_week_posted,day_type,days_between_experience_and_post,review_time,review_time_encoded
0,5,GB,Dest Iny,7,Not Verified,repeat,1,Asda,"ST2 9AL Had an issue with the scan, an employe...",st2 9al issue scan employee named rose helped ...,215,35,0.435000,4.0,0.8316,5,ST2 9AL,st2 9al,7,2,0,0,0,2024-11-11,2024-11-15 11:42:09,2024-11-15,11,10,Business Hours,4,Business Day,4,late_review,0
1,1,GB,Paula Gray,4,Not Verified,repeat,1,Asda,Your call centre staff are not the most helpfu...,call centre staff helpful several times tried ...,265,39,-0.228571,2.0,-0.5859,2,Your call centre staff are not the most…,call centre staff,17,3,0,0,0,2024-11-13,2024-11-13 18:26:52,2024-11-13,11,17,Business Hours,2,Business Day,0,quick_review,1
2,1,GB,Neil Dodds,4,Not Verified,repeat,1,Asda,"Just received my order from asda accrington, ...",received order asda accrington ordered 2 rump ...,236,36,0.250000,4.0,0.4939,4,Just received my order from asda…,received order asda,19,3,0,0,0,2024-11-14,2024-11-14 21:20:00,2024-11-14,11,20,Evening,3,Business Day,0,quick_review,1
3,1,GB,Julie Currey,5,Not Verified,repeat,1,Asda,"Just had an online home delivery dropped off, ...",online home delivery dropped substitutes items...,370,56,-0.275000,2.0,-0.9540,1,"Asda, where did it all go wrong?",asda go wrong,13,3,0,0,0,2024-11-13,2024-11-13 23:34:32,2024-11-13,11,22,Evening,2,Business Day,0,quick_review,1
4,2,GB,Jessiejames,3,Not Verified,repeat,1,Asda,Used the Asda rewards app to save for Christma...,used asda rewards app save christmas noticed p...,152,23,-0.033333,3.0,0.9071,5,Rewards not worth it,rewards worth,13,2,0,0,0,2024-10-01,2024-11-10 22:10:35,2024-11-10,11,21,Evening,6,Weekend,40,late_review,0


In [9]:
import os
os.makedirs('data', exist_ok=True)

csv_path = 'data/data_trustpilot_3.csv'
df_trustpilot.to_csv(csv_path, index=False)
print(f"Full DataFrame exported to {csv_path}")

train_path = 'data/train_trustpilot_3.csv'
test_path = 'data/test_trustpilot_3.csv'

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"Training set exported to {train_path}")
print(f"Test set exported to {test_path}")

X_train_tfidf = np.load('data/X_train_tfidf.npy')
X_test_tfidf = np.load('data/X_test_tfidf.npy')
feature_names = np.load('data/tfidf_features.npy', allow_pickle=True)

X_train_bow = np.load('data/X_train_bow.npy')
X_test_bow = np.load('data/X_test_bow.npy')
bow_features = np.load('data/bow_features.npy', allow_pickle=True)


Full DataFrame exported to data/data_trustpilot_3.csv
Training set exported to data/train_trustpilot_3.csv
Test set exported to data/test_trustpilot_3.csv
